In [33]:
import pandas as pd

In [34]:
def clean_person_nbr_id(df):
    df.loc[:, "person_nbr"] = (
        df.person_nbr.astype(str)
        .str.lower()
        .str.strip()
        .str.replace(r"\s+", "", regex=True)
    )
    return df


def read_tbl():
    dfa = pd.read_csv("../../../download/data/Vermont/Vermont_index.csv")

    dfa = dfa.pipe(clean_person_nbr_id)

    dfb = pd.read_csv(
        "../../../download/data/Vermont/vt-2023-original-employees.csv"
    )
    dfc = pd.read_csv(
        "../../../download/data/Vermont/vt-2023-original-employment.csv"
    )

    dfb = dfb.rename(columns={"emp_rec_no": "person_nbr"})
    dfc = dfc.rename(columns={"emp_rec_no": "person_nbr"})

    dfb = dfb.pipe(clean_person_nbr_id)
    dfc = dfc.pipe(clean_person_nbr_id)

    dfb = dfb[["person_nbr", "sex", "emp_id"]]

    dfc = dfc[["person_nbr", "action_description"]]

    df = pd.merge(dfa, dfb, on="person_nbr")
    df = pd.merge(df, dfc, on="person_nbr")
    df = df.drop(columns=["person_nbr", "middle_name", "age"])
    df = df.rename(
        columns={
            "action_description": "separation_reason",
            "emp_id": "person_nbr",
            "middle_initial": "middle_name",
        }
    )
    return df


def clean_misc(df):
    df.loc[:, "sex"] = (
        df.sex.astype(str)
        .fillna("")
        .str.lower()
        .str.strip()
        .str.replace(r"^m$", "Male", regex=True)
        .str.replace(r"^f$", "Female", regex=True)
    )

    df.loc[:, "suffix"] = df.suffix.str.replace(
        r"JR", "Jr", regex=True
    ).str.replace(r"SR", "Sr", regex=True)

    df.loc[:, "agency_name"] = df.agency_name.str.replace(
        r"OF", "Of", regex=True
    ).str.replace(r"U\.s\.", "U.S", regex=True)
    return df


df = read_tbl()

df = df.pipe(clean_misc)
df = df.fillna("")
df = df[~((df.person_nbr.fillna("") == ""))]

df

/var/folders/4r/jhc99db14v58mjmncgtry0kc0000gn/T/ipykernel_46667/2988154574.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['2' '2' '2' ... '31848' '31861' '31885']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, "person_nbr"] = df.person_nbr.astype(str).str.lower().str.strip().str.replace(r"\s+", "", regex=True)
/var/folders/4r/jhc99db14v58mjmncgtry0kc0000gn/T/ipykernel_46667/2988154574.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['2' '5' '35' ... '31908' '31909' '31910']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, "person_nbr"] = df.person_nbr.astype(str).str.lower().str.strip().str.replace(r"\s+", "", regex=True)
/var/folders/4r/jhc99db14v58mjmncgtry0kc0000gn/T/ipykernel_46667/2988154574.py:2: FutureWarning: Setting an ite

,full_name,last_name,first_name,middle_name,suffix,birth_year,type,agency_name,rank,class,start_date,end_date,sex,person_nbr,separation_reason
0,TIMOTHY P HANLEY,HANLEY,TIMOTHY,P,,1974.0,LAW ENFORCEMENT,MILTON POLICE DEPARTMENT,PATROL OFFICER,FULL-TIME,1998-02-22,2000-01-29,Male,TPH8934,APPOINTED
1,TIMOTHY P HANLEY,HANLEY,TIMOTHY,P,,1974.0,LAW ENFORCEMENT,MILTON POLICE DEPARTMENT,PATROL OFFICER,FULL-TIME,1998-02-22,2000-01-29,Male,TPH8934,TERMINATED
2,TIMOTHY P HANLEY,HANLEY,TIMOTHY,P,,1974.0,LAW ENFORCEMENT,MILTON POLICE DEPARTMENT,PATROL OFFICER,FULL-TIME,1998-02-22,2000-01-29,Male,TPH8934,APPOINTED
3,TIMOTHY P HANLEY,HANLEY,TIMOTHY,P,,1974.0,LAW ENFORCEMENT,MILTON POLICE DEPARTMENT,PATROL OFFICER,FULL-TIME,1998-02-22,2000-01-29,Male,TPH8934,APPOINTED
4,TIMOTHY P HANLEY,HANLEY,TIMOTHY,P,,1974.0,LAW ENFORCEMENT,MILTON POLICE DEPARTMENT,PATROL OFFICER,FULL-TIME,1998-02-22,2000-01-29,Male,TPH8934,DEPLOYED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35497,ROBERT P FOURNIER,FOURNIER,ROBERT,P,,1985.0,RECRUIT IN BASIC ACADEMY,ESSEX POLICE DEPARTMENT,RECRUIT,LEVEL III,2022-08-22,,Male,RPF6367,HIRED
35498,AKEEM J BROWN,BROWN,AKEEM,J,,1996.0,RECRUIT IN BASIC ACADEMY,BURLINGTON POLICE DEPARTMENT,RECRUIT,LEVEL III,2022-08-08,,Male,AJB7590,HIRED
35499,CATHLEEN D PIGEON,PIGEON,CATHLEEN,D,,1989.0,RECRUIT IN BASIC ACADEMY,SHELBURNE POLICE DEPARTMENT,PATROL OFFICER,LEVEL III,2022-08-22,,Female,CDP7825,HIRED
35500,JOSEPH T GRANT,GRANT,JOSEPH,T,,2000.0,RECRUIT IN BASIC ACADEMY,ST. JOHNSBURY POLICE DEPARTMENT,RECRUIT,LEVEL III,2022-08-01,,Male,JTG9288,HIRED


In [35]:
df.to_csv("../data/output/vermont_index.csv", index=False)